In [ ]:
from funcs.notebook import *

# Composite Table of confirmed exoplanets
path = "20_01_2021_confirmed_uncontroversial_exoplanet_systems.csv"

exokepler = pd.read_csv(f"../data/{path}") # composite table

# read in TESS-TOI sample 
path = "../data/2021_01_13_TESS_TOI_CATALOG.csv"


exotess = pd.read_csv(path, skiprows=4)

# rename the relevant columns for transit masking
exotess = exotess.rename(index=str, 
                         columns={'Transit Duration Value':"pl_trandur",
                                  'Orbital Period Value': "pl_orbper", 
                                  'Epoch Value':"pl_tranmidepoch"})


sep = "-----------------------------------------"

def mprint(message):
    print(sep)
    print(message)
    print(sep)
    
offset = {#"K2":2454833.,
          "Kepler":2454833.,
          "TESS":2457000., 
          }    



In [ ]:
facility = {"Cerro Tololo Inter-American Observatory":"BJD-TDB",
          "HATSouth":"BJD",
          "MEarth Project":"BJD",
           'Transiting Exoplanet Survey Satellite (TESS)':"BJD-TDB"}

In [ ]:
df = pd.read_csv("../results/2020_02_obsdurs.csv")
df

In [ ]:
# exokepler[(exokepler.hostname == "TIC 410214986")]
exotess[(exotess.TIC == 410214986)]

In [ ]:

for i, row in df.iloc[61:].iterrows():
    print(i)
    lc = pd.read_csv(f"../results/observedtimes/{row.ID}_{row.qcs}_{row.lc_n}_{row.mission}.csv")
    if row.ID[:3] == "TIC":
        system_tess = exotess[(exotess.TIC == int(row.ID[3:]))]

#     print(system_tess)
    # ID is unique, also ignore entries that have no transits
    # because there is nothing to mask (they are still searched for flares)
    system_kepler = exokepler[(exokepler.hostname == row.ID)]# &
 #                      (exokepler.discoverymethod == "Transit")]
    print(system_kepler, system_tess)
    try:
#         print(system_kepler)
        if system_kepler.shape[0] > 0:
            system_kepler["pl_tranmidepoch"] = (system_kepler.pl_tranmid -
                                                offset[row.mission])
        system = pd.concat([system_kepler, system_tess],ignore_index=True)
#         print(system_kepler)
#         print(system)
#         print(system_kepler.pl_tranmid_systemref.iloc[0],system.pl_tranmid.iloc[0])
        
#         print(facility[system.iloc[0].disc_facility])

    except KeyError:
        system = system_tess
#         print(system.pl_tranmidepoch.iloc[0],system_kepler.pl_tranmid_systemref.iloc[0])
#         print(facility[system.iloc[0].disc_facility])

    print(system)
    system = system.sort_values(by="pl_orbper",ascending=True).iloc[0]
    tmid = system.pl_tranmidepoch
#     print(tmid, system_kepler.pl_tranmid)
    orbper = system.pl_orbper
    tflare = lc.time
    phase = ((tflare - tmid) % orbper) / orbper

    lc["phase"] =phase
    
    lc.to_csv(f"../results/observedtimes/{row.ID}_{row.qcs}_{row.lc_n}_{row.mission}.csv",index=False)

# AU Mic phases

In [ ]:

lc = pd.read_csv(f"../results/observedtimes/AU Mic_1_0_TESS.csv")

#     print(system_tess)
# ID is unique, also ignore entries that have no transits
# because there is nothing to mask (they are still searched for flares)
system_kepler = exokepler[(exokepler.hostname == "AU Mic")]# &
#                      (exokepler.discoverymethod == "Transit")]

if system_kepler.shape[0] > 0:
    system_kepler["pl_tranmidepoch"] = (system_kepler.pl_tranmid -
                                        offset["TESS"])
system = system_kepler

system = system.sort_values(by="pl_orbper",ascending=True).iloc[0]
tmid = system.pl_tranmidepoch
#     print(tmid, system_kepler.pl_tranmid)
orbper = system.pl_orbper
tflare = lc.time
phase = ((tflare - tmid) % orbper) / orbper

lc["phase"] =phase
print(lc)
lc.to_csv(f"../results/observedtimes/AU Mic_1_0_TESS.csv",index=False)

In [ ]:
plt.plot(lc.time, phase);

In [ ]:
lc["phase"] =phase

In [ ]:
lc.head()

In [ ]:
lc.to_csv(f"../results/observedtimes/{row.ID}_{row.qcs}_{row.lc_n}_{row.mission}.csv",index=False)

In [ ]:
system.columns.values